In [1]:
import mqMBE as vq
import numpy as np
import networkx as nx
from time import time
vq.dataretriver.qibo.set_backend("numpy")

[Qibo 0.1.7|INFO|2022-07-13 12:17:24]: Using tensorflow backend on /device:CPU:0
[Qibo 0.1.7|WARNING|2022-07-13 12:17:24]: qibotf library was not found. `tf.einsum` will be used to apply gates. In order to install Qibo's high performance custom operators for TensorFlow please use `pip install qibotf`. Alternatively, consider installing the qibojit backend.
[Qibo 0.1.7|INFO|2022-07-13 12:17:25]: Using numpy backend on /CPU:0


Initialize the database with the preferred name

In [4]:
vq.dataretriver.Benchmarker.initialize_database('Test')

OperationalError: table Test already exists

Import the graph to cut and choose a seed

In [2]:
graph = nx.read_weighted_edgelist('w09_100.0')
seed = 1

Fix the given seed, and get the result of the cut using the BURER2002 method

In [3]:
np.random.seed(seed)
result_exact = vq.dataretriver.Benchmarker._get_exact_solution(seed, graph)

Get the adjacency matrix of the graph, and its maximum eigenvalue

In [4]:
adjacency_matrix, max_eigenvalue = vq.dataretriver.Benchmarker._graph_to_dict(graph)

Fix the circuit, given the type of entanglement, the number of qubits and layers

In [5]:
qubits = 9
layer = 4
entanglement = 'rotating'
circuit = vq.ansatz.var_form(qubits, layer, entanglement)

Initialise the parameters for the VQA

In [6]:
initial_parameters = np.pi * np.random.uniform(-1, 1, len(circuit.get_parameters(format='flatlist')))

Initialize the solver used to run the VQA

In [7]:
solver = vq.multibase.MultibaseVQA(circuit, adjacency_matrix, max_eigenvalue, hyperparameters=(1, 0.4))

Given the number of nodes to encodes in the qubits, the given compression, and some flags regarding how to encode, do the encoding.
By default, the observable used are the ones of the two body correlators of same pauli letter ( filling in order XX, YY and ZZ, to exhaustion).
Possible parameters:
* Compression: the compression rate used in the encoding, default is 2 (quadratic compression). A bigger compression will use increasingly many-body correlators to account for the extra compression (3 -> three bodies and so on).
* lower_order_terms: whether or not use observables involving also less "bodies" in the correlators. For instance: whether or not use also the single-qubits observables when using the quadratic compression.
* shuffle: whether or not assign the nodes to the observables in the order the observables are initialized, or at random.
* same_letter : whether or not use only observables of the same pauli letter, or pick random multi-body correlators involving the right number of qubits given the compression (so, also cross-terms).
* print_string: print the final set of pauli string used to encode the nodes of graph, in order of assigment ( 0 = Identity, 1 = X, 2 = Y, 3 = Z)


In [8]:
nodes_number = 100
compression = 2
pauli_string_length = qubits
solver.encode_nodes(nodes_number, pauli_string_length, compression=compression,lower_order_terms=False, shuffle=False, seed=seed, same_letter=True, print_strings=True)

[(1, 1, 0, 0, 0, 0, 0, 0, 0), (1, 0, 1, 0, 0, 0, 0, 0, 0), (1, 0, 0, 1, 0, 0, 0, 0, 0), (1, 0, 0, 0, 1, 0, 0, 0, 0), (1, 0, 0, 0, 0, 1, 0, 0, 0), (1, 0, 0, 0, 0, 0, 1, 0, 0), (1, 0, 0, 0, 0, 0, 0, 1, 0), (1, 0, 0, 0, 0, 0, 0, 0, 1), (0, 1, 1, 0, 0, 0, 0, 0, 0), (0, 1, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 1, 0, 0, 0, 0), (0, 1, 0, 0, 0, 1, 0, 0, 0), (0, 1, 0, 0, 0, 0, 1, 0, 0), (0, 1, 0, 0, 0, 0, 0, 1, 0), (0, 1, 0, 0, 0, 0, 0, 0, 1), (0, 0, 1, 1, 0, 0, 0, 0, 0), (0, 0, 1, 0, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 1, 0, 0, 0), (0, 0, 1, 0, 0, 0, 1, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0), (0, 0, 1, 0, 0, 0, 0, 0, 1), (0, 0, 0, 1, 1, 0, 0, 0, 0), (0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 1, 0, 0, 1, 0, 0), (0, 0, 0, 1, 0, 0, 0, 1, 0), (0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 0, 0, 1, 1, 0, 0, 0), (0, 0, 0, 0, 1, 0, 1, 0, 0), (0, 0, 0, 0, 1, 0, 0, 1, 0), (0, 0, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 1, 0, 0), (0, 0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 0, 0, 0, 1, 1, 0), (0, 0, 0, 0, 

1.0

Set the activation function.

In [45]:
solver.set_activation(np.tanh)

Fix the bounds for the parameters of the circuit.

In [46]:
bounds = [(-np.pi, np.pi) for i in range(len(initial_parameters))]
cons = None

Run the optimization

In [47]:
my_time = time()
result, cut, parameters, extra, unrounded_solution, solution = solver.minimize(initial_parameters, method='SLSQP', bounds=bounds, tol=1e-03, constraints=cons,options={'maxiter': 1000000000})
timing = time() - my_time

Fix the type of the results, and load a row to the database.

In [60]:
epochs = extra['nfev']
max_energy, min_energy, number_parameters, initial_parameters, parameters, unrounded_solution, solution = cut, 'None', len(initial_parameters), str(initial_parameters.tolist()), str(parameters.tolist()), str(unrounded_solution), str(solution)

energy_ratio = (cut) / (result_exact[0][0])
activation_function_name = 'tanh'
instance = 'w09_100.0'
row = {'kind': 'MqMBE', 'instance': str(instance), 'trial': 0, 'layer_number': layer,
'nodes_number': nodes_number, 'optimization': 'SLSQP',
'activation_function': str(activation_function_name),
'compression': compression, 'pauli_string_length': pauli_string_length,
'entanglement': entanglement,
'graph_kind': 'Given', 'qubits': qubits, 'solution': solution,
'unrounded_solution': unrounded_solution,
'max_energy': max_energy, 'min_energy': min_energy, 'energy_ratio': energy_ratio,
'initial_parameters': initial_parameters, 'parameters': parameters,
'number_parameters': number_parameters, 'hyperparameter': str('(1,0.4)'),
'epochs': epochs, 'time': timing}
vq.datamanager.insert_value_table('Test', 'Test', row)

AttributeError: 'str' object has no attribute 'tolist'